# Disciplina de Classificação e Pesquisa de Dados
 
# Laboratório #4
 
### Implementação (em Python) de estratégias e funções relacionados com estruturas Hash

O objetivo consiste em usar os mecanismos de dicionário oferecidos pela linguagem Python 3 ou superior para armazenar e consultar informações, além de verificarmos o funcionamento de algumas técnicas de tratamento de colisões e de redimencionamento de tabelas. Em seguida, vamos implementar algumas funções de geração de códigos hash para diferentes tipos de dados.

Se possível, estude como Python implementa internamente um dicionário. Também avalie que tipos de estruturas semelhantes outras linguagens oferecem (p.ex., hash tables em Java/C# e arrays associativos em C++).  

----------------
## I) Python Dicts

Python oferece uma estrutura de dados muito utilizada chamada de dicionário. Essa estrutura é implementada internamente usando as técnicas de hash que vimos em aula.

Seguem alguns exemplos abaixo.

In [1162]:
# cria um dicionário vazio:
dicionario1 = {}

# cria um dicionário cujas chaves são números
dicionario2 = {1: 'bananas', 2: 'maças', 5: 'peras'}

# dicionário com chaves alfanuméricas (ou mistas):
dicionario3 = {'nome': 'Leonardo', 1: [2, 4, 3]}

# criando usando compreensions:
odd_squares = {x: x*x for x in range(11) if x%2 == 1}
print(odd_squares)

{1: 1, 3: 9, 5: 25, 7: 49, 9: 81}


In [1163]:
# Adicionando itens no dicionário:
dicionario1['leandro'] = {'status':'professor', 'lotacao': 'CINTED'}

In [1164]:
# consulta ao dicionário:
print(dicionario1['leandro'])

print(dicionario2[1])

print(dicionario3['nome'])

print(dicionario3[1])

print(odd_squares)

{'status': 'professor', 'lotacao': 'CINTED'}
bananas
Leonardo
[2, 4, 3]
{1: 1, 3: 9, 5: 25, 7: 49, 9: 81}


In [1165]:
# removendo um item:
dicionario3.pop(1)
print(dicionario3[1]) # erro porque a chave não existe mais!

KeyError: 1

In [ ]:
# apagando tudo em um dicionário:
dicionario2.clear()
print(dicionario2[1]) # erro porque não tem mais nada!

---------------
## 2) SimpleHashTable

Agora vamos analisar uma implementação simples de tabelas hash que usam a função módulo para gerar código hash (posição/endereço em uma tabela ou array) e a técnica de sondagem linear (linear probing) para tratar colisões.

Observe o código abaixo e seu teste de uso com diferentes exemplos.

In [1122]:
class SimpleHashTable:
    # construtor, basta passar tamanho inicial da tabela hash
    def __init__(self, size):
        self.size = size
        self.dicionario = [-1] * size                 # -1 significa que o local está vago
        self.conteudo   = [None] * size 
        self.used       = [False] * size                 
        
    def add(self, chave, dado):
        posicao = posicao_inicial = chave % self.size # usa o módulo
        
        if self.dicionario[posicao] == -1:            # se estiver vazio
            self.dicionario[posicao] = chave          # coloca a chave
            self.conteudo[posicao] = dado                      # associa dado
            self.used[posicao] = True
            return posicao
        else:                                         # se estiver ocupado, tenta achar lugar usando linear probing
            first_pass = True
            while posicao != posicao_inicial or first_pass:
                first_pass = False
                posicao = (posicao + 1) % self.size   # incrementa posição (mas fica dentro do intervalo do array de chaves)
                if self.dicionario[posicao] == -1:    
                    self.dicionario[posicao] = chave  
                    self.conteudo[posicao] = dado
                    self.used[posicao] = True
                    return posicao
                    
        if posicao == posicao_inicial:                  # se posicao igual à inicial é porque fez a volta e não achou
            return -1                                  # informa que deu problema (está cheio)
        else:   
            return posicao                             # retorna posição onde colocou o dado
    
    def get(self, chave):
        posicao_inicial = posicao = chave % self.size
        first_pass = True
        
        # busca elemento usando linear probing:
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            return self.conteudo[posicao]
        else:
            return None # se chegou aqui é porque não existe a chave
    
    def remove(self, chave):
        posicao_inicial = posicao = chave % self.size
        first_pass = True
        
        # busca elemento usando linear probing:
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            self.dicionario[posicao] = -1
            self.conteudo[posicao] = None
            return posicao
        else:
            return None # se chegou aqui é porque não existe a chave
        return -1
        
    # modifica tamanho da tabela criando nova tabela
    def resize_on_new(self, novo_tamanho):
        nova = SimpleHashTable(novo_tamanho) # cria nova tabela com novo tamanho
        for i in range(1,self.size): # passa todo os registros da tabela antiga para a nova, levando em conta nova função de hash
            if self.dicionario[i] != -1:
                nova.add(self.dicionario[i], self.conteudo[i])
        return nova

    # modifica tamanho da tabela nela mesmo
    def resize_in_situ(self, novo_tamanho):
        # guarda todas as infos da lista em auxiliares
        dicioaux = self.dicionario
        contaux = self.conteudo
        usedaux = self.used
        tam = self.size

        # aumenta o tamanho da tabela
        self = SimpleHashTable(novo_tamanho)
        # copia as infos de volta
        for i in range(0,tam):
            self.dicionario[i] = dicioaux[i]
            self.conteudo[i] = contaux[i]
            self.used[i] = usedaux[i]
        
        # para todo registro não nulo antigo na lista, remove e adiciona novamente pra realocar
        for i in (0,tam):
            if self.dicionario[i] != None:
                pos = self.remove(self.dicionario[i])
                self.add(self.dicionario[i], self.conteudo[i])
        return self
    
    def print(self):
        for indice in range(0, self.size):
            print(f"({indice:03d})[{self.dicionario[indice]:03d}] = {str(self.conteudo[indice]):15s} ({self.used[indice]})")

Abaixo veja alguns exemplos de uso:

In [1123]:
#cria uma estrutura com 20 posições
hash_table = SimpleHashTable(20)

In [1124]:
# mostra estrutura interna da hashtable
hash_table.print()

(000)[-01] = None            (False)
(001)[-01] = None            (False)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[-01] = None            (False)
(011)[-01] = None            (False)
(012)[-01] = None            (False)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1125]:
# adiciona uma chave e dados associados:
hash_table.add(10, "valor 10")
hash_table.print() # mostra como ficou a estrutura interna

(000)[-01] = None            (False)
(001)[-01] = None            (False)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[010] = valor 10        (True)
(011)[-01] = None            (False)
(012)[-01] = None            (False)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1126]:
hash_table.add(20, "valor 20")
hash_table.print()

(000)[020] = valor 20        (True)
(001)[-01] = None            (False)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[010] = valor 10        (True)
(011)[-01] = None            (False)
(012)[-01] = None            (False)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1127]:
hash_table.add(0, "valor 0")
hash_table.print()

(000)[020] = valor 20        (True)
(001)[000] = valor 0         (True)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[010] = valor 10        (True)
(011)[-01] = None            (False)
(012)[-01] = None            (False)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1128]:
hash_table.add(12, "valor 12")
hash_table.print()

(000)[020] = valor 20        (True)
(001)[000] = valor 0         (True)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[010] = valor 10        (True)
(011)[-01] = None            (False)
(012)[012] = valor 12        (True)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1129]:
# recupera o dado associado com a chave 0:
print(hash_table.get(0))

valor 0


In [1130]:
# recupera dado associado com chave 10:
print(hash_table.get(10))

valor 10


In [1131]:
# remove elemento de chave 10:
print(hash_table.remove(10))

10


In [1132]:
hash_table.print()

(000)[020] = valor 20        (True)
(001)[000] = valor 0         (True)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[-01] = None            (True)
(011)[-01] = None            (False)
(012)[012] = valor 12        (True)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)


In [1133]:
print(hash_table.get(10)) # removeu e portanto não retorna nada

None


Vamos agora analisar a diferença entre usar um número primo como tamanho para a tabela ou outro número qualquer.

O primeiro exemplo adiciona 4 números que possuem divisores comuns (entre si e entre o tamanho da tabela). Perceba que há concentração e todos caem na mesma posição.

O segundo exemplo usa um número primo, aumentando as chances de não ocorrer concentração, visto que não há mais propriedades comuns (divisores, no caso) entre os números e o tamanho da tabela.

In [1134]:
# cria dicionario com tamanho não primo
hash_table = SimpleHashTable(10)
hash_table.add(10, 10)
hash_table.add(20, 20)
hash_table.add(30, 30)
hash_table.add(40, 40)
hash_table.print()

(000)[010] = 10              (True)
(001)[020] = 20              (True)
(002)[030] = 30              (True)
(003)[040] = 40              (True)
(004)[-01] = None            (False)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[-01] = None            (False)
(008)[-01] = None            (False)
(009)[-01] = None            (False)


In [1135]:
# cria dicionario com tamanho  primo
hash_table = SimpleHashTable(13)
hash_table.add(10, 10)
hash_table.add(20, 20)
hash_table.add(30, 30)
hash_table.add(40, 40)
hash_table.print()

(000)[-01] = None            (False)
(001)[040] = 40              (True)
(002)[-01] = None            (False)
(003)[-01] = None            (False)
(004)[030] = 30              (True)
(005)[-01] = None            (False)
(006)[-01] = None            (False)
(007)[020] = 20              (True)
(008)[-01] = None            (False)
(009)[-01] = None            (False)
(010)[010] = 10              (True)
(011)[-01] = None            (False)
(012)[-01] = None            (False)


Via de regra, números primos são tipicamente usados para oferecer certa uniformidade às colisões quando a população de chaves apresenta certas características matemáticas (tais como serem múltiplos de determinado número).

Se o número de buckets (entradas na tabela) e a polulação de chaves compartilham um fator comum, todas as chaves da população irão ser mapeadas para um bucket que seja múltiplo desse fator comum. 

------------------------
Agora vamos encher a tabela!

Perceba que alguns números e dados não serão inseridos...

In [1136]:
# perceba que alguns
for i in range(1,20):
    if hash_table.add(i, f"valor {i}") != -1:
        print(f"valor {i} inserido com sucesso")
    else:
        print(f"ERRO ao inserir valor {i}")

valor 1 inserido com sucesso
valor 2 inserido com sucesso
valor 3 inserido com sucesso
valor 4 inserido com sucesso
valor 5 inserido com sucesso
valor 6 inserido com sucesso
valor 7 inserido com sucesso
valor 8 inserido com sucesso
valor 9 inserido com sucesso
ERRO ao inserir valor 10
ERRO ao inserir valor 11
ERRO ao inserir valor 12
ERRO ao inserir valor 13
ERRO ao inserir valor 14
ERRO ao inserir valor 15
ERRO ao inserir valor 16
ERRO ao inserir valor 17
ERRO ao inserir valor 18
ERRO ao inserir valor 19


In [1137]:
hash_table.print()

(000)[009] = valor 9         (True)
(001)[040] = 40              (True)
(002)[001] = valor 1         (True)
(003)[002] = valor 2         (True)
(004)[030] = 30              (True)
(005)[003] = valor 3         (True)
(006)[004] = valor 4         (True)
(007)[020] = 20              (True)
(008)[005] = valor 5         (True)
(009)[006] = valor 6         (True)
(010)[010] = 10              (True)
(011)[007] = valor 7         (True)
(012)[008] = valor 8         (True)


-----------------
Exercício 1
====

A classe SimpleHashTable fornecida pelo professor tem dois métodos vazios: 

- resize_on_new(): aumenta ou diminui a tabela, usando uma nova região de memória 
- resize_in_situ(): aumenta ou diminui a tabela, usando a mesma região de memória

Elas servem, justamente, para você atualizar o tamanho da tabela quando ela fica cheia (aumentando-a) ou quando está muito vazia (diminuindo-a).

Implemente essas duas funções. Depois, use-as para atualizar o tamanho da estrutura e execute novamente o código que insere 20 valores. Mostre que suas funções (de resize on new ou in situ) funcionam. 

In [1138]:
hash_table = hash_table.resize_in_situ(40)
hash_table.print()

(000)[-01] = None            (True)
(001)[040] = 40              (True)
(002)[001] = valor 1         (True)
(003)[002] = valor 2         (True)
(004)[030] = 30              (True)
(005)[003] = valor 3         (True)
(006)[004] = valor 4         (True)
(007)[020] = 20              (True)
(008)[005] = valor 5         (True)
(009)[006] = valor 6         (True)
(010)[010] = 10              (True)
(011)[007] = valor 7         (True)
(012)[008] = valor 8         (True)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)
(020)[-01] = None            (False)
(021)[-01] = None            (False)
(022)[-01] = None            (False)
(023)[-01] = None            (False)
(024)[-01] = None            (False)
(025)[-01] = None            (False)
(026)[-01] = None            (False)
(027)[-01] = N

In [1139]:
hash_table = hash_table.resize_on_new(30)
hash_table.print()

(000)[030] = 30              (True)
(001)[001] = valor 1         (True)
(002)[002] = valor 2         (True)
(003)[003] = valor 3         (True)
(004)[004] = valor 4         (True)
(005)[005] = valor 5         (True)
(006)[006] = valor 6         (True)
(007)[007] = valor 7         (True)
(008)[008] = valor 8         (True)
(009)[-01] = None            (False)
(010)[040] = 40              (True)
(011)[010] = 10              (True)
(012)[-01] = None            (False)
(013)[-01] = None            (False)
(014)[-01] = None            (False)
(015)[-01] = None            (False)
(016)[-01] = None            (False)
(017)[-01] = None            (False)
(018)[-01] = None            (False)
(019)[-01] = None            (False)
(020)[020] = 20              (True)
(021)[-01] = None            (False)
(022)[-01] = None            (False)
(023)[-01] = None            (False)
(024)[-01] = None            (False)
(025)[-01] = None            (False)
(026)[-01] = None            (False)
(027)[-01] = 

------------
## Funções de hash para outros tipos de dados

Nossa SimpleHashTable foi feita de maneira a aceitar somente números inteiros como chaves. Mas, como vimos nos exemplos de Python, outros tipos de chave poderiam ser utilizados.

Vamos criar (e testar) uma função que recebe uma string e a transforma em um número:

In [1140]:
# recebe uma string e devolve um numero que é o valor acumulado de seus códigos ASCII
def hash_str1(string):
    acc = 0
    for i in range(0, len(string)):
        acc = acc + ord(string[i])        
    return acc

In [1141]:
hash_str1("aaaa")

388

In [1142]:
hash_str1("abab")

390

In [1143]:
hash_str1("baab")

390

In [1144]:
hash_str1("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz")

12078

Perceba que essa função tem 2 problemas:

1. toda chave que possui a mesma combinação de caracteres, em posições diferentes, gera a mesma chave!
2. como os endereços gerados correspondem à soma de valores, essa soma pode ter diferentes tamanhos (casas). Não há um limite para seu tamanho, estando essa soma limitada ao tamanho dos números inteiros.

O primeiro problema pode gerar colisões e seria interessante resolvê-lo

O segundo problema não é tão grave, pois podemos aplicar uma segunda função hash que mapeia os números inteiros gerados para uma faixa de valores (tal como fizemos com a função módulo nos primeiros exemplos). Mas a função módulo pode gerar colisões demais e seria interessante termos uma função que evita isso já na geração do primeiro valor.

Além disso, se desejássemos usar outros tipos de dados como chave, teríamos que pensar em uma função para cada tipo, cada uma com suas respectivas heurísticas para gerar números com pouca repetição e com controle de tamanho. O ideal é que toda chave gerasse uma sequencia de bits de tamanho fixo, independente do seu tipo. 

Vamos ver como fazer isso em Python. 

A função 'bin()' converte um número em uma sequencia de bits (em uma string):

In [1145]:
bin(10)

'0b1010'

In [1146]:
bin(166)

'0b10100110'

Mas ela inclui o tipo da sequencia no início da string (veja que todos possuem '0b' no início)... Python faz isso para diferenciar sequencias de bits de sequencias em outras bases. Em hexadecimal o início seria '0h').

Mas isso é fácil de resolver:

In [1147]:
#devolve uma sugstring iniciando na posição 2:
bin(10)[2:]

'1010'

Mas a função 'bin()' não funciona para strings... :-(

In [ ]:
# bin nao funciona para strings!
bin("aaaa")

Como sabemos programar, vamos criar uma que faça isso para nós! ;-)

In [1148]:
def str_to_bin(string):
    resultado = ""
    for c in string:
        resultado = resultado + bin(ord(c))[2:]
    return resultado   

In [1149]:
str_to_bin("aaaa")

'1100001110000111000011100001'

Podemos agora lidar como problema do tamanho. Uma maneira de limitar o tamanho é considerarmos todos os elementos como sendo sequencias de 8bit e combinarmos essas sequencias usando operadores booleanos. 

O exemplo seguinte combina dois-a-dois os caracteres de uma string, gerando sequencias de 8 bits:

In [1150]:
# não há função XOR para strings, então vamos criar uma:

# _xormap é um dicionário de combinações de bits e seu resultado em 'xor'
_xormap = {('0','1'):'1', ('1','0'):'1', ('1','1'):'0', ('0','0'):'0'}

# usa uma list compreensions para agrupar duas strings em uma lista de pares ordenados (zip)
# depois, para cada par (a,b) aplica o _xormap:
def xor(x, y):
    return ''.join([_xormap[a,b] for a,b in zip(x,y)])

In [1151]:
# devolve uma sequencia de bits de tamanho fixo
def hash_str2(string):
    resultado = str_to_bin(string[0])
    for i in range(1, len(string)): # do segundo em diante
        str_tmp = str_to_bin(string[i])
        resultado = xor(resultado, str_tmp) # combina 2 a 2 usando Xor (bit a bit)
    return resultado

Vamos testá-la:

In [1152]:
hash_str2("aaaaaa")

'0000000'

In [1153]:
hash_str2("abaaaa")

'0000011'

In [1154]:
hash_str2("abaaadddddddddda")

'0000011'

Mas ainda temos o problema de sequências que possuem os mesmos caracteres em posições diferentes gerarem o mesmo código:

In [1155]:
hash_str2("aabaaa")

'0000011'

-----------------
Exercício 2
====

1. Crie uma Hash Table que aceite Strings como chaves, a StringHashTable, que gera um número a partir da combinação dos bytes dos caracteres de uma string. Faça com que ela gere sequencias ou números diferentes para sequencias que tenham o mesmo conjunto de letras em posições diferentes (i.e., "ABCD" == "BADC" == "DCBA" ....). Observe os slides da disciplina para um insight de como isso poderia ser feito (hashing parte 2).

2. E se precisássemos adicionar qualquer coisa como chave, como em Python, o que fazer? Tente criar uma estrutura que aceite qualquer coisa como chave: AnyHashTable.

In [1156]:
class StringHashTable:
    def __init__(self, size):
        self.size = size
        self.dicionario = [None] * size                 # None significa que o local está vago
        self.conteudo   = [None] * size 
        self.used       = [False] * size                 
        
    # função recursiva que codifica uma string em natural 
    # usando hashing polinomial com constante a = 3;
    # recebe a string (chave), a posição atual e a posição final da string
    def codifica(self,chave,pos_curr,pos_fim):
        if pos_curr == pos_fim:
            return ord(chave[pos_curr])
        else:
            return ord(chave[pos_curr]) + (3 * (codifica(chave,(pos_curr+1),pos_fim)))

    def add(self, chave, dado):
        # codifica a string em número
        codigo = self.codifica(chave,0,len(chave)-1)
        posicao = posicao_inicial = codigo % self.size # usa o módulo a partir do número obtido
        
        if self.dicionario[posicao] == None:            
            self.dicionario[posicao] = chave          
            self.conteudo[posicao] = dado                
            self.used[posicao] = True
            return posicao
        else:                                         
            first_pass = True
            while posicao != posicao_inicial or first_pass:
                first_pass = False
                posicao = (posicao + 1) % self.size   
                if self.dicionario[posicao] == None:    
                    self.dicionario[posicao] = chave  
                    self.conteudo[posicao] = dado
                    self.used[posicao] = True
                    return posicao
                    
        if posicao == posicao_inicial:                
            return -1                                 
        else:   
            return posicao                            
    
    def print(self):
        for indice in range(0, self.size):
            print(f"({indice})[{self.dicionario[indice]}] = {str(self.conteudo[indice])} ({self.used[indice]})")    
    
    def get(self, chave):
        codigo = self.codifica(chave,0,len(chave)-1)
        posicao_inicial = posicao = codigo % self.size
        first_pass = True
        
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            return self.conteudo[posicao]
        else:
            return None 
    
    def remove(self, chave):
        codigo = self.codifica(chave,0,len(chave)-1)
        posicao_inicial = posicao = codigo % self.size
        first_pass = True
        
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            self.dicionario[posicao] = None
            self.conteudo[posicao] = None
            return posicao
        else:
            return None 
        return -1
        
    def resize_on_new(self, novo_tamanho):
        nova = StringHashTable(novo_tamanho)
        for i in range(1,self.size):
            if self.dicionario[i] != None:
                nova.add(self.dicionario[i], self.conteudo[i])
        return nova

    def resize_in_situ(self, novo_tamanho):
        dicioaux = self.dicionario
        contaux = self.conteudo
        usedaux = self.used
        tam = self.size

        self = StringHashTable(novo_tamanho)
        for i in range(0,tam):
            self.dicionario[i] = dicioaux[i]
            self.conteudo[i] = contaux[i]
            self.used[i] = usedaux[i]
        
        for i in (0,tam):
            if self.dicionario[i] != None:
                pos = self.remove(self.dicionario[i])
                self.add(self.dicionario[i], self.conteudo[i])
            
        return self
    

In [1157]:
string_table = StringHashTable(21)
string_table.add('nome','marthyna')
string_table.add('idade','20')
string_table.add('sexo','feminino')
string_table.add('cor','vermelho')
string_table.print()

print('\n',string_table.get('nome'))
print(string_table.remove('sexo'),'\n')
string_table = string_table.resize_on_new(17)
string_table.print()
print('\n')
string_table = string_table.resize_in_situ(21)
string_table.print()


(0)[None] = None (False)
(1)[None] = None (False)
(2)[nome] = marthyna (True)
(3)[None] = None (False)
(4)[None] = None (False)
(5)[None] = None (False)
(6)[idade] = 20 (True)
(7)[None] = None (False)
(8)[None] = None (False)
(9)[None] = None (False)
(10)[None] = None (False)
(11)[None] = None (False)
(12)[cor] = vermelho (True)
(13)[None] = None (False)
(14)[None] = None (False)
(15)[None] = None (False)
(16)[None] = None (False)
(17)[None] = None (False)
(18)[None] = None (False)
(19)[sexo] = feminino (True)
(20)[None] = None (False)

 marthyna
19 

(0)[None] = None (False)
(1)[nome] = marthyna (True)
(2)[None] = None (False)
(3)[None] = None (False)
(4)[cor] = vermelho (True)
(5)[None] = None (False)
(6)[None] = None (False)
(7)[None] = None (False)
(8)[None] = None (False)
(9)[None] = None (False)
(10)[None] = None (False)
(11)[None] = None (False)
(12)[None] = None (False)
(13)[None] = None (False)
(14)[idade] = 20 (True)
(15)[None] = None (False)
(16)[None] = None (False)


(0)[N

In [1158]:
class AnyHashTable:
    def __init__(self, size):
        self.size = size
        self.dicionario = [None] * size                 
        self.conteudo   = [None] * size 
        self.used       = [False] * size                 
        
    # função recursiva que codifica tipos iterativos e strings em natural
    # recebe a chave e de acordo com o tipo do elemento dentro da lista/string, 
    # usa codificação polinomial com a função ord ou não
    def codifica(self,chave,pos_curr,pos_fim):
        if pos_curr == pos_fim:
            if isinstance(chave[pos_curr], str):
                return ord(chave[pos_curr])
            else:
                return chave[pos_curr]
        else:
            if isinstance(chave[pos_curr], str):
                return ord(chave[pos_curr]) + (3 * (self.codifica(chave,(pos_curr+1),pos_fim)))
            else:
                return chave[pos_curr] + (3 * (self.codifica(chave,(pos_curr+1),pos_fim)))

    def add(self, chave, dado):
        posicao = posicao_inicial = 0
        # leva em consideração o tipo da chave:
        # string/lista/tupla = hashing polinomial
        # números = pula codificação e vai pro módulo direto
        if (isinstance(chave, str)) or (isinstance(chave, list)) or (isinstance(chave, tuple)):
            codigo = self.codifica(chave,0,len(chave)-1) 
            posicao = posicao_inicial = codigo % self.size 
        else:
            posicao = posicao_inicial = chave % self.size 
        
        if self.dicionario[posicao] == None:            
            self.dicionario[posicao] = chave          
            self.conteudo[posicao] = dado             
            self.used[posicao] = True
            return posicao
        else:                                         
            first_pass = True
            while posicao != posicao_inicial or first_pass:
                first_pass = False
                posicao = (posicao + 1) % self.size   
                if self.dicionario[posicao] == None:    
                    self.dicionario[posicao] = chave  
                    self.conteudo[posicao] = dado
                    self.used[posicao] = True
                    return posicao
                    
        if posicao == posicao_inicial:                
            return -1                                 
        else:   
            return posicao                            
    
    def print(self):
        for indice in range(0, self.size):
            print(f"({indice})[{self.dicionario[indice]}] = {str(self.conteudo[indice])} ({self.used[indice]})")    
    
    def get(self, chave):
        posicao = posicao_inicial = 0
        if (isinstance(chave, str)) or (isinstance(chave, list)) or (isinstance(chave, tuple)):
            codigo = self.codifica(chave,0,len(chave)-1) 
            posicao = posicao_inicial = codigo % self.size 
        else:
            posicao = posicao_inicial = chave % self.size 
        
        first_pass = True
        
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            return self.conteudo[posicao]
        else:
            return None
    
    def remove(self, chave):
        posicao = posicao_inicial = 0
        if (isinstance(chave, str)) or (isinstance(chave, list)) or (isinstance(chave, tuple)):
            codigo = self.codifica(chave,0,len(chave)-1) 
            posicao = posicao_inicial = codigo % self.size 
        else:
            posicao = posicao_inicial = chave % self.size 
        
        first_pass = True
        
        while self.dicionario[posicao] != chave and self.used and (posicao_inicial != posicao or first_pass):
            first_pass = False
            posicao = (posicao + 1) % self.size
            
        if self.dicionario[posicao] == chave:
            self.dicionario[posicao] = None
            self.conteudo[posicao] = None
            return posicao
        else:
            return None 
        return -1
        
    def resize_on_new(self, novo_tamanho):
        nova = AnyHashTable(novo_tamanho)
        for i in range(1,self.size):
            if self.dicionario[i] != None:
                nova.add(self.dicionario[i], self.conteudo[i])
        return nova

    def resize_in_situ(self, novo_tamanho):
        dicioaux = self.dicionario
        contaux = self.conteudo
        usedaux = self.used
        tam = self.size

        self = AnyHashTable(novo_tamanho)
        for i in range(0,tam):
            self.dicionario[i] = dicioaux[i]
            self.conteudo[i] = contaux[i]
            self.used[i] = usedaux[i]
        
        for i in (0,tam):
            if self.dicionario[i] != None:
                pos = self.remove(self.dicionario[i])
                self.add(self.dicionario[i], self.conteudo[i])
            
        return self
    

In [1161]:
any_table = AnyHashTable(21)
any_table.add('nome','marthyna')
any_table.add(20,'20')
any_table.add((1,2,3,4),'feminino')
any_table.add(['a','b','c','d'],'vermelho')
any_table.print()

print('\n',any_table.get((1,2,3,4)))
print(any_table.remove(['a','b','c','d']),'\n')
any_table = any_table.resize_on_new(17)
any_table.print()
print('\n')
any_table = any_table.resize_in_situ(21)
any_table.print()


(0)[None] = None (False)
(1)[None] = None (False)
(2)[None] = None (False)
(3)[None] = None (False)
(4)[None] = None (False)
(5)[None] = None (False)
(6)[None] = None (False)
(7)[None] = None (False)
(8)[None] = None (False)
(9)[None] = None (False)
(10)[None] = None (False)
(11)[None] = None (False)
(12)[None] = None (False)
(13)[['a', 'b', 'c', 'd']] = vermelho (True)
(14)[nome] = marthyna (True)
(15)[None] = None (False)
(16)[(1, 2, 3, 4)] = feminino (True)
(17)[None] = None (False)
(18)[None] = None (False)
(19)[None] = None (False)
(20)[20] = 20 (True)

 feminino
13 

(0)[None] = None (False)
(1)[None] = None (False)
(2)[None] = None (False)
(3)[nome] = marthyna (True)
(4)[20] = 20 (True)
(5)[None] = None (False)
(6)[(1, 2, 3, 4)] = feminino (True)
(7)[None] = None (False)
(8)[None] = None (False)
(9)[None] = None (False)
(10)[None] = None (False)
(11)[None] = None (False)
(12)[None] = None (False)
(13)[None] = None (False)
(14)[None] = None (False)
(15)[None] = None (False)
(16)[